# is_cancelled Prediction- (classification ) - Tensorflow


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler

X_train_norm_clf_pca = pd.read_pickle('X_train_norm_clf_pca.pkl')
X_test_norm_clf_pca = pd.read_pickle('X_test_norm_clf_pca.pkl')
y_train_clf = pd.read_pickle('y_train_clf.pkl')
y_test_clf = pd.read_pickle('y_test_clf.pkl')
#X_train_norm_clf_pca.head()
#y_test_clf.head()

In [12]:
"""convert to numpy.ndarray"""
X_train_arr=X_train_norm_clf_pca.values 
X_test_arr=X_test_norm_clf_pca.values
y_train_arr=y_train_clf.values
y_test_arr=y_test_clf.values

type(X_train_arr)

numpy.ndarray

In [13]:
"""Train an Multilayer Perceptron (MLP) with TensorFlow - Keras API"""

import tensorflow as tf


model = tf.keras.models.Sequential()
tf.keras.layers.BatchNormalization()
model.add(tf.keras.layers.Dense(256, input_shape=(70,), activation='relu',use_bias=False))
model.add(tf.keras.layers.Dropout(0.2))
tf.keras.layers.BatchNormalization()
model.add(tf.keras.layers.Dense(128,activation='relu',use_bias=False))
tf.keras.layers.BatchNormalization()
model.add(tf.keras.layers.Dense(16, activation='softmax',use_bias=False))

model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=1e-3), metrics=["accuracy"])

print(model.summary())



model.fit(X_train_arr, y_train_arr, batch_size=1, epochs=5)

model.evaluate(X_test_arr,y_test_arr)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               17920     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32768     
_________________________________________________________________
dense_9 (Dense)              (None, 16)                2048      
Total params: 52,736
Trainable params: 52,736
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
75170/75170 [==============================] - 122s 2ms/sample - loss: 0.4646 - acc: 0.7861
Epoch 2/5
75170/75170 [==============================] - 134s 2ms/sample - loss: 0.3806 - acc: 0.8148
Epoch 3/5
75170/75170 [==============================] - 135s

[0.35812658967243455, 0.82943785]

In [14]:
#Results from various Activation function - [loss function , Acuracy]

"""
relu-  [0.3609906734423616, 0.8394947]
selu - [0.38405117149714463, 0.81708413]
elu -  [0.380795744237877, 0.8159357]
tanh - [0.39610912362878786, 0.81432164]

relu (optimized)- [0.3583572881965955, 0.82940686]

"""    


'\nrelu-  [0.3609906734423616, 0.8394947]\nselu - [0.38405117149714463, 0.81708413]\nelu -  [0.380795744237877, 0.8159357]\ntanh - [0.39610912362878786, 0.81432164]\n\nrelu (optimized)- [0.3583572881965955, 0.82940686]\n\n'